In [3]:
# ========================================
# PARTE 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS
# ========================================

import os
import re
import csv
import math
import json
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from pathlib import Path
from typing import List, Tuple, Dict, Any

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, classification_report, roc_curve, auc)
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline

warnings.filterwarnings('ignore', category=UserWarning)
plt.switch_backend("Agg")  # garante salvar figuras sem precisar de display


try:
    from tqdm.std import TqdmWarning
    warnings.filterwarnings('ignore', category=TqdmWarning)
except Exception:
    pass

RANDOM_STATE = 42


DATASET_PATH = "IMDB Dataset.csv"
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Amostragens para etapas custosas (ajuste conforme sua máquina)
LDA_SAMPLE = 5000         # documentos usados para selecionar N de tópicos (None = usar todos)
TSNE_SAMPLE = 2000        # documentos usados no t-SNE
SHAP_SAMPLE = 200         # documentos para SHAP (apenas uma amostra, para não ficar pesado)
LIME_INDEX = 0            # índice do exemplo de teste para LIME/Force-plot

# Utilidades
def _find_text_and_label_columns(df: pd.DataFrame) -> Tuple[str, str]:
    cols = {c.lower(): c for c in df.columns}

    text_candidates = ["review", "texto", "text", "content", "comentario"]
    label_candidates = ["sentiment", "label", "rotulo", "classe", "target"]
    text_col = next((cols[c] for c in text_candidates if c in cols), None)
    label_col = next((cols[c] for c in label_candidates if c in cols), None)
    if not text_col or not label_col:
        raise ValueError(f"Não encontrei colunas de texto/label. Colunas disponíveis: {list(df.columns)}")
    return text_col, label_col

def _normalize_sentiment(series: pd.Series) -> pd.Series:
    # mapeia 'positive'/'negative' -> 1/0, ou tenta converter automaticamente
    s = series.astype(str).str.lower().str.strip()
    mapping = {"positive": 1, "negativo": 0, "negative": 0, "pos": 1, "neg": 0, "1": 1, "0": 0}
    out = s.map(lambda x: mapping.get(x, None))
    # se ainda tiver None, tenta converter para int
    if out.isna().any():
        try:
            out2 = s.astype(int)
            out = out.fillna(out2)
        except Exception:
            pass
    if out.isna().any():
        # ultimo recurso: binariza assumindo duas classes distintas
        uniq = sorted(s.unique())
        if len(uniq) == 2:
            out = s.map({uniq[0]: 0, uniq[1]: 1})
    if out.isna().any():
        raise ValueError("Não foi possível normalizar a coluna de sentimento/label para 0/1.")
    return out.astype(int)

def _basic_clean(txt: str) -> str:
    txt = txt or ""
    txt = re.sub(r"<br\s*/?>", " ", txt, flags=re.I)  # remove <br/>
    txt = re.sub(r"[^A-Za-z0-9\s']", " ", txt)       # mantém letras, números e apóstrofo
    txt = re.sub(r"\s+", " ", txt).strip().lower()
    return txt

# Carrega dataset
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Arquivo não encontrado: {DATASET_PATH}. Ajuste a variável DATASET_PATH no topo do notebook.")

df = pd.read_csv(DATASET_PATH, encoding="utf-8")
text_col, label_col = _find_text_and_label_columns(df)

df = df[[text_col, label_col]].dropna()
df[text_col] = df[text_col].astype(str).map(_basic_clean)
df[label_col] = _normalize_sentiment(df[label_col])

# Embaralha para reduzir viés
df = df.sample(frac=1.0, random_state=RANDOM_STATE).reset_index(drop=True)

X = df[text_col].values
y = df[label_col].values

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

print(f"Total de documentos: {len(df)} | Treino: {len(X_train_text)} | Teste: {len(X_test_text)}")

# ========================================
# PARTE 2: CRIAÇÃO DAS FEATURES (TF-IDF)
# ========================================

tfidf = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1, 2),
    min_df=5,
    stop_words='english'
)
X_train = tfidf.fit_transform(X_train_text)
X_test  = tfidf.transform(X_test_text)

print(f"Dimensão TF-IDF: {X_train.shape}")

# ========================================
# PARTE 3: MODELAGEM DE TÓPICOS COM LDA (com busca de N tópicos por coerência/perplexidade)
# ========================================

# Para LDA, é recomendado usar contagens (CountVectorizer)
count_vect = CountVectorizer(
    max_features=30000,
    ngram_range=(1, 1),
    min_df=5,
    stop_words='english'
)
# Usa amostra para acelerar
lda_texts = X_train_text[:LDA_SAMPLE] if (LDA_SAMPLE and LDA_SAMPLE < len(X_train_text)) else X_train_text
counts = count_vect.fit_transform(lda_texts)

# Tenta usar gensim para coerência (c_v). Se não disponível, usa perplexidade como proxy.
HAS_GENSIM = False
try:
    import gensim
    import gensim.corpora as corpora
    from gensim.models.coherencemodel import CoherenceModel
    HAS_GENSIM = True
except Exception as e:
    print("[AVISO] gensim indisponível. A seleção de tópicos usará perplexidade como proxy.")

def _tokenize_for_gensim(texts: List[str]) -> List[List[str]]:
    return [re.findall(r"[a-zA-Z']+", t.lower()) for t in texts]

def select_n_topics(counts_mat, count_vect, texts, topic_range=(5, 10, 15, 20)) -> Tuple[int, Dict[int, float]]:
    from sklearn.decomposition import LatentDirichletAllocation
    scores = {}
    if HAS_GENSIM:
        tokenized = _tokenize_for_gensim(texts)
        dictionary = corpora.Dictionary(tokenized)
        for nt in range(topic_range[0], topic_range[-1] + 1, 5):
            lda = LatentDirichletAllocation(n_components=nt, random_state=RANDOM_STATE, learning_method="batch", max_iter=10)
            lda.fit(counts_mat)
            # top palavras por tópico
            vocab = np.array(count_vect.get_feature_names_out())
            topics = []
            for i in range(nt):
                comp = lda.components_[i]
                top_idx = np.argsort(comp)[::-1][:20]
                topics.append(list(vocab[top_idx]))
            cm = CoherenceModel(topics=topics, texts=tokenized, dictionary=dictionary, coherence='c_v')
            coherence = float(cm.get_coherence())
            scores[nt] = coherence
            print(f"[coherence c_v] tópicos={nt}: {coherence:.4f}")
        best_n = max(scores.items(), key=lambda kv: kv[1])[0]
    else:
        for nt in range(topic_range[0], topic_range[-1] + 1, 5):
            lda = LatentDirichletAllocation(n_components=nt, random_state=RANDOM_STATE, learning_method="batch", max_iter=10)
            lda.fit(counts_mat)
            perp = float(lda.perplexity(counts_mat))
            # Menor perplexidade é melhor; para padronizar, salvamos -perp como "score"
            scores[nt] = -perp
            print(f"[perplexity proxy] tópicos={nt}: perplexity={perp:.2f}")
        best_n = max(scores.items(), key=lambda kv: kv[1])[0]
    return best_n, scores

best_n_topics, topic_scores = select_n_topics(counts, count_vect, list(lda_texts))

from sklearn.decomposition import LatentDirichletAllocation
lda_final = LatentDirichletAllocation(n_components=best_n_topics, random_state=RANDOM_STATE, learning_method="batch", max_iter=15)
# Ajusta em amostra para velocidade
lda_final.fit(counts)

# Extrai top palavras por tópico
vocab_cv = np.array(count_vect.get_feature_names_out())
TOPK_WORDS = 15
topics_words = []
for k in range(best_n_topics):
    comp = lda_final.components_[k]
    top_idx = np.argsort(comp)[::-1][:TOPK_WORDS]
    topics_words.append((k, list(vocab_cv[top_idx])))

# Salva tópicos em CSV para referência
pd.DataFrame([{"topic": k, "top_words": ", ".join(words)} for k, words in topics_words]).to_csv(OUTPUT_DIR / "lda_topics.csv", index=False)

print(f"Melhor número de tópicos: {best_n_topics}")
print("Exemplo de tópicos (top palavras):")
for k, words in topics_words[: min(5, len(topics_words))]:
    print(f"  Tópico {k}: {', '.join(words)}")

# ========================================
# PARTE 4: CLASSIFICAÇÃO DE TEXTOS (com busca de hiperparâmetros)
# ========================================

# Usaremos TF-IDF + Regressão Logística
base_clf = LogisticRegression(max_iter=2000, solver='liblinear', class_weight=None, random_state=RANDOM_STATE)
param_grid = {
    "logisticregression__C": [0.1, 0.5, 1.0, 2.0, 5.0],
    "logisticregression__penalty": ["l2"]
}

pipeline = make_pipeline(
    TfidfVectorizer(
        max_features=50000, ngram_range=(1,2), min_df=5, stop_words='english'
    ),
    base_clf
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    n_jobs=-1,
    cv=cv,
    verbose=1
)
grid.fit(X_train_text, y_train)

best_pipeline = grid.best_estimator_
print(f"Melhor pipeline: {best_pipeline}")
print(f"Melhor F1 (validação): {grid.best_score_:.4f}")

# ========================================
# PARTE 5: AVALIAÇÃO DE DESEMPENHO (Precisão, Recall, F1, AUC-ROC)
# ========================================

y_pred = best_pipeline.predict(X_test_text)
# Para AUC, precisamos de probabilidade da classe positiva
try:
    y_proba = best_pipeline.predict_proba(X_test_text)[:, 1]
except Exception:
    # fallback se o classificador não suportar predict_proba
    # usa decisão como score (não calibrado)
    y_proba = best_pipeline.decision_function(X_test_text)
    # normaliza para [0,1]
    y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min() + 1e-9)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc_roc = roc_auc_score(y_test, y_proba)

print("\n=== MÉTRICAS DE TESTE ===")
print(f"Acurácia:  {acc:.4f}")
print(f"Precisão:  {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"AUC-ROC:   {auc_roc:.4f}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred, digits=4))

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Curva ROC - Classificador (Teste)')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "roc_curve.png", dpi=150)
plt.close()

# ========================================
# PARTE 6: t-SNE PARA VISUALIZAÇÃO DOS AGRUPAMENTOS
# ========================================

# Para t-SNE em texto, é comum reduzir com SVD antes (para ~50 dims) e depois aplicar t-SNE
svd = TruncatedSVD(n_components=50, random_state=RANDOM_STATE)
X_all = tfidf.fit_transform(df[text_col].values)  # usa TF-IDF (fora do pipeline p/ visualizar todo o dataset)
X_all_svd = svd.fit_transform(X_all)

# Amostra para t-SNE (p/ performance)
tsne_idx = np.arange(len(df))
if TSNE_SAMPLE and TSNE_SAMPLE < len(df):
    rng = np.random.default_rng(RANDOM_STATE)
    tsne_idx = rng.choice(len(df), size=TSNE_SAMPLE, replace=False)

X_tsne_input = X_all_svd[tsne_idx]
y_tsne = df[label_col].values[tsne_idx]

# Compatibilidade de versões: algumas versões usam `n_iter`, outras `max_iter` (prefira max_iter p/ evitar FutureWarning)
from inspect import signature
tsne_kwargs = dict(n_components=2, perplexity=30, random_state=RANDOM_STATE, init='pca', learning_rate='auto')
sig = signature(TSNE.__init__)
if 'max_iter' in sig.parameters:
    tsne_kwargs['max_iter'] = 1000
elif 'n_iter' in sig.parameters:
    tsne_kwargs['n_iter'] = 1000

tsne = TSNE(**tsne_kwargs)
X_2d = tsne.fit_transform(X_tsne_input)

plt.figure(figsize=(7, 6))
scatter = plt.scatter(X_2d[:,0], X_2d[:,1], c=y_tsne, s=8, alpha=0.7)
plt.title("t-SNE dos Documentos (cores = rótulos de sentimento)")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "tsne_plot.png", dpi=150)
plt.close()

print(f"t-SNE salvo em: {OUTPUT_DIR / 'tsne_plot.png'}")

# ========================================
# PARTE 7: EXPLICABILIDADE (LIME, SHAP, FORCE-PLOT) + ANÁLISE DOS RESULTADOS
# ========================================

# Checagem/instalação automática de dependências opcionais (LIME/SHAP)
try:
    import importlib, sys, subprocess
    def _ensure_module(mod_name: str, pip_name: str = None, version: str = None) -> bool:
        try:
            importlib.import_module(mod_name)
            return True
        except Exception:
            try:
                pkg = pip_name or mod_name
                spec = f"{pkg}=={version}" if version else pkg
                print(f"[INFO] Instalando pacote '{spec}' (auto)...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", spec])
                importlib.import_module(mod_name)
                return True
            except Exception as _e:
                print(f"[AVISO] Não foi possível instalar/importar '{mod_name}': {_e}")
                return False
    _ensure_module("lime", "lime", "0.2.0.1")
    _ensure_module("shap", "shap", "0.46.0")
except Exception:
    # se a checagem falhar por algum motivo, segue o fluxo normal; os blocos abaixo ainda tratarão exceções
    pass

# ---- LIME ----
LIME_OK = True
try:
    from lime.lime_text import LimeTextExplainer
    class_names = ["negativo", "positivo"]
    explainer = LimeTextExplainer(class_names=class_names)
    def predict_proba_text(texts: List[str]):
        # usa o best_pipeline encontrado (TF-IDF + LR)
        try:
            return best_pipeline.predict_proba(texts)
        except Exception:
            # se não houver predict_proba, tenta decision_function com normalização
            scores = best_pipeline.decision_function(texts)
            scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)
            # retorna proba para duas classes
            probs = np.vstack([1 - scores, scores]).T
            return probs
    sample_text = X_test_text[LIME_INDEX if 0 <= LIME_INDEX < len(X_test_text) else 0]
    exp = explainer.explain_instance(sample_text, predict_proba_text, num_features=10, labels=[0,1])
    lime_path = OUTPUT_DIR / "lime_sample.html"
    exp.save_to_file(str(lime_path))
    print(f"LIME salvo em: {lime_path}")
except Exception as e:
    LIME_OK = False
    print(f"[AVISO] LIME não pôde ser executado: {e}")

# ---- SHAP ----
SHAP_OK = True
try:
    import shap

    # Extrai componentes do pipeline
    tfidf_vec = None
    logreg = None
    for step_name, step_est in best_pipeline.named_steps.items():
        if isinstance(step_est, TfidfVectorizer):
            tfidf_vec = step_est
        if isinstance(step_est, LogisticRegression):
            logreg = step_est
    if tfidf_vec is None or logreg is None:
        raise RuntimeError("Não encontrei TfidfVectorizer ou LogisticRegression no pipeline final.")

    X_train_tfidf = tfidf_vec.transform(X_train_text)
    # amostra para SHAP
    shap_idx = np.arange(X_train_tfidf.shape[0])
    if SHAP_SAMPLE and SHAP_SAMPLE < len(shap_idx):
        rng = np.random.default_rng(RANDOM_STATE)
        shap_idx = rng.choice(len(shap_idx), size=SHAP_SAMPLE, replace=False)
    X_bg = X_train_tfidf[shap_idx]

    # 1º tentamos API antiga com 'interventional' (compatível com muitas instalações)
    explainer = None
    shap_values = None
    try:
        explainer = shap.LinearExplainer(logreg, X_bg, feature_perturbation="interventional")
        X_test_tfidf = tfidf_vec.transform(X_test_text[:SHAP_SAMPLE if SHAP_SAMPLE else 200])
        shap_values = explainer.shap_values(X_test_tfidf)
        expected_value = explainer.expected_value
        api_mode = "legacy"
    except Exception as e_old:
        # 2º fallback: nova API com masker
        try:
            masker = shap.maskers.Independent(X_bg)
            explainer = shap.Explainer(logreg, masker)
            X_test_tfidf = tfidf_vec.transform(X_test_text[:SHAP_SAMPLE if SHAP_SAMPLE else 200])
            explanation = explainer(X_test_tfidf)
            shap_values = explanation  # Explanation object
            expected_value = explanation.base_values.mean() if hasattr(explanation, "base_values") else explainer.expected_value
            api_mode = "masker"
        except Exception as e_new:
            raise RuntimeError(f"Falha no SHAP (antigo e novo): {e_old} | {e_new}")

    feature_names = tfidf_vec.get_feature_names_out()

    # summary plot
    plt.figure()
    if api_mode == "legacy":
        shap.summary_plot(shap_values, X_test_tfidf, feature_names=feature_names, show=False)
    else:
        shap.summary_plot(shap_values, feature_names=feature_names, show=False)
    shap_summary_path = OUTPUT_DIR / "shap_summary.png"
    plt.tight_layout()
    plt.savefig(shap_summary_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"SHAP summary salvo em: {shap_summary_path}")

    # force plot para um exemplo
    shap_force_path = OUTPUT_DIR / "shap_force.html"
    if api_mode == "legacy":
        i_force = 0
        plot_obj = shap.force_plot(expected_value, shap_values[i_force,:], matplotlib=False)
        shap.save_html(str(shap_force_path), plot_obj)
    else:
        # usa o primeiro exemplo do Explanation
        plot_obj = shap.plots.force(shap_values[0], matplotlib=False)
        shap.save_html(str(shap_force_path), plot_obj)
    print(f"SHAP force-plot salvo em: {shap_force_path}")

except Exception as e:
    SHAP_OK = False
    print(f"[AVISO] SHAP não pôde ser executado: {e}")

# ---- ANÁLISE AUTOMÁTICA (conclusões) ----
# 1) Resumo de métricas
conclusoes = []
conclusoes.append(f"(1) Classificador final: {type(best_pipeline.named_steps['logisticregression']).__name__} com TF-IDF; F1_teste={f1:.4f}, AUC={auc_roc:.4f}")

# 2) Principais features (coeficientes) do modelo
try:
    logreg = best_pipeline.named_steps["logisticregression"]
    vec = best_pipeline.named_steps["tfidfvectorizer"]
    coefs = logreg.coef_[0]
    idx_top_pos = np.argsort(coefs)[-15:][::-1]
    idx_top_neg = np.argsort(coefs)[:15]
    top_pos_words = [vec.get_feature_names_out()[i] for i in idx_top_pos]
    top_neg_words = [vec.get_feature_names_out()[i] for i in idx_top_neg]
    conclusoes.append("(2) Palavras mais pró-positivo (amostras): " + ", ".join(top_pos_words[:10]))
    conclusoes.append("(3) Palavras mais pró-negativo (amostras): " + ", ".join(top_neg_words[:10]))
except Exception:
    conclusoes.append("(2-3) Não foi possível extrair coeficientes/top palavras.")

# 3) LDA: número de tópicos e exemplo de tópicos
conclusoes.append(f"(4) LDA selecionou {best_n_topics} tópicos (por {'coerência c_v' if HAS_GENSIM else 'perplexidade-proxy'}).")
if topics_words:
    ex = ";  ".join([f"T{k}: " + ", ".join(words[:5]) for k, words in topics_words[:min(3, len(topics_words))]])
    conclusoes.append(f"(5) Tópicos (exemplo): {ex}")

# 4) t-SNE: apenas indicativo visual
conclusoes.append("(6) t-SNE salvo em outputs/tsne_plot.png para inspeção visual de separação/aglomerados.")

# 5) LIME/SHAP status
conclusoes.append(f"(7) LIME {'ok' if LIME_OK else 'falhou'}; SHAP {'ok' if SHAP_OK else 'falhou'}. Consulte a pasta outputs/.")

print("\n=== CONCLUSÕES (Geradas automaticamente) ===")
for c in conclusoes:
    print("-", c)
print("\nArquivos gerados na pasta 'outputs/': tsne_plot.png, roc_curve.png, lda_topics.csv, shap_summary.png, shap_force.html, lime_sample.html (quando disponíveis).")


Total de documentos: 50000 | Treino: 40000 | Teste: 10000
Dimensão TF-IDF: (40000, 50000)
[AVISO] gensim indisponível. A seleção de tópicos usará perplexidade como proxy.
[perplexity proxy] tópicos=5: perplexity=3420.09
[perplexity proxy] tópicos=10: perplexity=3662.32
[perplexity proxy] tópicos=15: perplexity=3901.24
[perplexity proxy] tópicos=20: perplexity=4120.65
Melhor número de tópicos: 5
Exemplo de tópicos (top palavras):
  Tópico 0: film, story, love, life, great, man, time, young, war, best, world, good, movie, wife, does
  Tópico 1: film, good, great, story, love, series, like, time, just, really, best, characters, episode, version, better
  Tópico 2: movie, just, film, like, bad, really, good, don, time, movies, acting, make, seen, plot, watch
  Tópico 3: film, story, films, characters, like, movie, character, just, director, way, time, horror, man, scenes, good
  Tópico 4: movie, like, people, just, time, think, film, funny, good, movies, watch, really, great, love, life
Fi

C:\Users\Master\miniconda3\Lib\site-packages\shap\explainers\_linear.py:95: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)


SHAP summary salvo em: outputs\shap_summary.png
SHAP force-plot salvo em: outputs\shap_force.html

=== CONCLUSÕES (Geradas automaticamente) ===
- (1) Classificador final: LogisticRegression com TF-IDF; F1_teste=0.9051, AUC=0.9660
- (2) Palavras mais pró-positivo (amostras): excellent, great, best, perfect, amazing, wonderful, brilliant, hilarious, favorite, superb
- (3) Palavras mais pró-negativo (amostras): worst, awful, waste, bad, boring, terrible, disappointment, worse, dull, poor
- (4) LDA selecionou 5 tópicos (por perplexidade-proxy).
- (5) Tópicos (exemplo): T0: film, story, love, life, great;  T1: film, good, great, story, love;  T2: movie, just, film, like, bad
- (6) t-SNE salvo em outputs/tsne_plot.png para inspeção visual de separação/aglomerados.
- (7) LIME ok; SHAP ok. Consulte a pasta outputs/.

Arquivos gerados na pasta 'outputs/': tsne_plot.png, roc_curve.png, lda_topics.csv, shap_summary.png, shap_force.html, lime_sample.html (quando disponíveis).
